In [39]:
from IPython.display import display
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [40]:
df = pd.read_csv('loan_stat542.zip', compression='zip')

In [41]:
tdf = pd.read_csv('Project3_test_id.csv').astype(int)

In [42]:
with pd.option_context('display.max_columns', None):
    display(df.head(4))

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,zip_code,addr_state,dti,earliest_cr_line,fico_range_low,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,38098114,15000,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Fully Paid,debt_consolidation,Debt consolidation,235xx,VA,12.03,Aug-1994,750,754,6,0,138008.0,29.0,17,w,Individual,0.0,0.0
1,36805548,10400,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Charged Off,credit_card,Credit card refinancing,937xx,CA,14.92,Sep-1989,710,714,17,0,6133.0,31.6,36,w,Individual,1.0,0.0
2,37842129,21425,60 months,15.59,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.0,Source Verified,Fully Paid,credit_card,Credit card refinancing,658xx,MO,18.49,Aug-2003,685,689,10,0,16374.0,76.2,35,w,Individual,0.0,0.0
3,37662224,7650,36 months,13.66,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000.0,Source Verified,Charged Off,debt_consolidation,Debt consolidation,850xx,AZ,34.81,Aug-2002,685,689,11,0,16822.0,91.9,20,f,Individual,0.0,0.0


We need to create categories for *emp_title* and *title*, for these we could use fuzzy string algorithm and merge some of those title. This could be last. 

Somehow we need to handle *zip_code* as well, there is definetely a correlation between fully paid loans and metropolitan areas but I'm unsure how to handle this to see the correlatio you can run this line:

```df[['zip_code', 'loan_status']].groupby(['zip_code', 'loan_status']).size()[:25]```

*earliest_cr_line* should be stripped from months.

In [43]:
import datetime 
year = datetime.datetime.now().year
df.earliest_cr_line = df.earliest_cr_line.apply(lambda x: year - int(x[-4:]))

In [44]:
CAT_COLS = ["term", "grade", "sub_grade", "emp_length", "home_ownership", 
            "verification_status", "purpose", 
            "initial_list_status", "application_type"]

In [45]:
df = df.drop(columns=["emp_title", "zip_code", "title", "addr_state"])

In [46]:
df = pd.get_dummies(df, columns=CAT_COLS) #, drop_first=True)

In [47]:
df['loan_status'] = np.where(df['loan_status']=="Fully Paid", 0, 1)

In [48]:
df = df.fillna(df.mean())

In [49]:
testidx = df.id.isin(tdf.test3)
trainidx = ~df.id.isin(tdf.test3)

In [50]:
mu = df.mean()
std = df.std()
df = (df - mu) / std
min_val = np.abs(df.min().min())
df.loan_status = (df.loan_status * std.loan_status) + mu.loan_status 

In [51]:
X = df.loc[:, df.columns != 'loan_status'].values
y = df['loan_status'].values

In [52]:
X.shape

(844006, 98)

In [53]:
X_train = X[trainidx]
X_test = X[testidx]
y_train = y[trainidx]
y_test = y[testidx]

In [386]:
dfTest = df[testidx]
dfTrain = df[trainidx]
X_train = dfTrain.loc[:, dfTrain.columns != 'loan_status'].values
y_train = dfTrain.loan_status.values
X_test = dfTest.loc[:, dfTest.columns != 'loan_status'].values
y_test = dfTest.loan_status.values

In [298]:
clf = LogisticRegression(random_state=0, solver='lbfgs', 
                         multi_class='multinomial', n_jobs=-1).fit(X_train, y_train)

In [25]:
clf = GradientBoostingClassifier(random_state=0)

In [35]:
clf = MLPClassifier(hidden_layer_sizes=(25,), alpha=0.000001, activation='relu', max_iter=250)

In [36]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-06, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=250, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [37]:
np.mean(y_test == clf.predict(X_test))

0.8033846761453397

In [60]:
log_loss(y_test, prob)

0.44593102273666707

In [59]:
pd.read_csv('mysubmission2.txt', header=None).iloc[:,[1,2]].values

In [321]:
xgb = XGBClassifier(n_jobs=8, n_estimators=250, booster='gbtree',  max_depth=3, reg_alpha=0.001)
xgb.fit(X_train, y_train)
log_loss(y_test, xgb.predict_proba(X_test))

0.44582002067018206